# Dog Breed Recognition Project

## Project Basics

#### Problem

Our goal is to identify dog breed from a photo of the dog.  
The project is taken from [Kaggle Dog Breed Identification Competition](https://www.kaggle.com/c/dog-breed-identification/data).  
The machine learning problem is **supervised learning > multiclass classification**.  
Our task is to build a neural network image classifier using TensorFlow and TensorFlow Hub.

#### Evaluation

The evaluation metric set for the competition is Multiclass Log Loss.  
Our target matrix contains N Dogs x M Breeds, true breed = 1, rest = 0.  
Our model predicts a probability matrix with the same dimensions.  
Multiclass Log Loss measures the error of model predictions (the lower the better).  
Muticlass Log Loss is applied in image classification, natural language processing, and recommendation systems.

#### Data Source

Data is acquired from [Kaggle Dog Breed Identification Competition](https://www.kaggle.com/c/dog-breed-identification/data).

#### Features / Data Dictionary

Our model analyzes image files (unstructured data) > deep learning / transfer learning.  
There are 120 unique dog breeds in the training set > multiclass classification with 120 classes.  
There are 10 222 images in the training set.  
There are 10 357 images in the test set.

## Importing Libraries

In [ ]:
### importing tensorflow
import tensorflow as tflow
print(tflow.__version__)
print(tflow.config.list_physical_devices())

### importing tensorflow hub
import tensorflow_hub as thub
print(thub.__version__)

### python libraries
from pathlib import Path
from math import ceil
from datetime import datetime

### external libraries
import numpy
from pandas import read_csv, DataFrame, concat
from matplotlib import pyplot
from IPython.display import Image

## Importing Data

In [ ]:
### importing labels
labels_df = read_csv(filepath_or_buffer="data-files-labels.csv")

In [ ]:
### exploring labels: head
labels_df.head()

In [ ]:
### exploring labels: info
labels_df.info()

In [ ]:
### exploring labels: unique breeds
unique_breeds = labels_df["breed"].to_numpy()
unique_breeds = numpy.unique(ar=unique_breeds)
len(unique_breeds)

In [ ]:
### exploring labels: mean of images/breed
round(number=labels_df["breed"].value_counts().mean(), ndigits=3)

Google recommends at least 10 images per class.  
We have adequate data with ~85 images per class on average.

## Preparing Data

#### Creating Image Filepaths

In [ ]:
### counting number of images in train folder
image_list = [image for image in Path("D:/WorkDev_GitHub/projectData/dataDogRecognition/train").iterdir()]
len(image_list)

In [ ]:
### creating image filepaths from image ids
labels_df["imagepath"] = "D:/WorkDev_GitHub/projectData/dataDogRecognition/train/" + labels_df["id"] + ".jpg"

In [ ]:
### exploring imagepaths: head
labels_df.head()

In [ ]:
### exploring imagepaths: info
labels_df.info()

In [ ]:
### exploring imagepaths: checking validity of random imagepath
print(labels_df.loc[9000, "breed"])
print()
print(labels_df.loc[9000, "imagepath"])
print()
Image(filename=labels_df.loc[9000, "imagepath"])

#### Reducing and Splitting

In [ ]:
### dataframe inits
fTrain_df = DataFrame()
fValid_df = DataFrame()
rTrain_df = DataFrame()
rValid_df = DataFrame()

In [ ]:
### creating train and valid dataframes
for breed in unique_breeds:
    work_df: DataFrame = DataFrame()
    work_df = labels_df.loc[labels_df["breed"] == breed]
    work_df = work_df.sample(n=work_df.index.size, random_state=42)
    train_num = int(0.9 * work_df.index.size)
    valid_num = work_df.index.size - train_num
    fTrain_df = concat(objs=[fTrain_df, work_df.iloc[:train_num]])
    fValid_df = concat(objs=[fValid_df, work_df.iloc[train_num:]])
    work_df = work_df.sample(n=12, random_state=42)
    rTrain_df = concat(objs=[rTrain_df, work_df.iloc[:10]])
    rValid_df = concat(objs=[rValid_df, work_df.iloc[10:]])

In [ ]:
### shuffling train dataframes
fTrain_df = fTrain_df.sample(n=fTrain_df.index.size, random_state=42)
rTrain_df = rTrain_df.sample(n=rTrain_df.index.size, random_state=42)

In [ ]:
### verifying dimensions of train and valid dataframes
fTrain_df.shape, fValid_df.shape, rTrain_df.shape, rValid_df.shape

#### Creating Datasets

All machine learning algorithms require data in numerical format.  
So the first task is to turn images and labels into tensors.  
A tensor is a numerical matrix with n-dimensions, like a numpy ndarray.

In [ ]:
### function creating image tensors
def imageTensor(aInput_df=DataFrame(), aImage_size=224):
    """
    Converts image files into constant image tensors.\n
    Combines individual image tensors into a tensor array.
    """
    tensor_array = numpy.empty(shape=(0,224,224,3), dtype=numpy.float32)
    for _,row in aInput_df.iterrows():
        image_tensor = tflow.io.read_file(filename=row["imagepath"])
        image_tensor = tflow.image.decode_jpeg(contents=image_tensor, channels=3)
        image_tensor = tflow.image.convert_image_dtype(image=image_tensor, dtype=tflow.float32)
        image_tensor = tflow.image.resize(images=image_tensor, size=[aImage_size,aImage_size])
        tensor_array = numpy.append(arr=tensor_array, values=numpy.array([image_tensor]), axis=0)
    return tflow.constant(value=tensor_array)

In [ ]:
### function creating label tensors
def labelTensor(aInput_df=DataFrame()):
    """
    Encodes true dog breeds into constant label tensors.\n
    Combines individual label tensors into a tensor array.
    """
    tensor_array = numpy.empty(shape=(0,120), dtype=numpy.int8)
    for _,row in aInput_df.iterrows():
        label_tensor = numpy.zeros(shape=120, dtype=numpy.int8)
        label_tensor[numpy.where(unique_breeds == row["breed"])] = 1
        tensor_array = numpy.append(arr=tensor_array, values=numpy.array([label_tensor]), axis=0)
    return tflow.constant(value=tensor_array)

In [ ]:
### function creating dataset
def createDataset(tInput_df=DataFrame()):
    """
    Creates image and label tensors.\
    Combines tensors into a keras dataset.
    """
    counter = 1
    num_shards = ceil(tInput_df.index.size / 256)
    for index in range(0, tInput_df.index.size, 256):
        print(f"Batch {counter} / {num_shards}\r", end="")
        image_tensors = imageTensor(aInput_df=tInput_df.iloc[index:index+256], aImage_size=224)
        label_tensors = labelTensor(aInput_df=tInput_df.iloc[index:index+256])
        if index == 0:
            full_dataset = tflow.data.Dataset.from_tensor_slices(tensors=(image_tensors,label_tensors))
        else:
            shard_dataset = tflow.data.Dataset.from_tensor_slices(tensors=(image_tensors,label_tensors))
            full_dataset = full_dataset.concatenate(dataset=shard_dataset)
        counter += 1  
    return full_dataset

In [ ]:
### creating full train dataset
fTrain_dataset = createDataset(tInput_df=fTrain_df)
fTrain_dataset.element_spec

In [ ]:
### creating full valid dataset
fValid_dataset = createDataset(tInput_df=fValid_df)
fValid_dataset.element_spec

In [ ]:
### creating reduced train dataset
rTrain_dataset = createDataset(tInput_df=rTrain_df)
rTrain_dataset.element_spec

In [ ]:
### creating reduced valid dataset
rValid_dataset = createDataset(tInput_df=rValid_df)
rValid_dataset.element_spec

#### Visualizing Datasets

In [ ]:
### function extracting first 32 image/label tensors from dataset
def extractTensors(aDataset=tflow.data.Dataset.from_tensor_slices([])):
    images = numpy.empty(shape=(0,224,224,3), dtype=numpy.float32)
    labels = numpy.empty(shape=(0,120), dtype=numpy.int8)
    for image,label in aDataset.as_numpy_iterator():
        images = numpy.append(arr=images, values=[image], axis=0)
        labels = numpy.append(arr=labels, values=[label], axis=0)
        if len(images) == 32: break
    return images, labels

In [ ]:
### function visualizing 32 image/label tensors
def visualizeTensors(aImages=numpy.array([]), aLabels=numpy.array([])):
    """
    Displays 32 image/label tensor pairs from a tensorflow dataset.
    """
    pyplot.figure(figsize=(8,12))
    for index,image,label in zip(range(1, 33), aImages, aLabels):
        label: numpy.ndarray
        pyplot.subplot(8, 4, index)
        pyplot.imshow(X=image)
        pyplot.title(label=unique_breeds[label.argmax()], fontsize=8)
        pyplot.axis("off")
    return

In [ ]:
### visualizing full train dataset
images,labels = extractTensors(aDataset=fTrain_dataset)
visualizeTensors(aImages=images, aLabels=labels)

In [ ]:
### visualizing full valid dataset
images,labels = extractTensors(aDataset=fValid_dataset)
visualizeTensors(aImages=images, aLabels=labels)

In [ ]:
### visualizing reduced train dataset
images,labels = extractTensors(aDataset=rTrain_dataset)
visualizeTensors(aImages=images, aLabels=labels)

In [ ]:
### visualizing valid dataset
images,labels = extractTensors(aDataset=rValid_dataset)
visualizeTensors(aImages=images, aLabels=labels)

## Preparing Neural Network

#### Creating Neural Network

**Optimal network parameters:**  
for binary classification: sigmoid (activation), binary crossentropy (loss)  
for multiclass classification: softmax (activation), categorical crossentropy (loss)

In [ ]:
### setting base network features
INPUT_SHAPE = [None, 224, 224, 3] # batch, height, width, colorchannel
MODEL_URL = "https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow2/130-224-classification/2"
OUTPUT_SHAPE = len(unique_breeds) # number of classes (breeds)
NUM_EPOCHS = 100

In [ ]:
### function creating neural network
def createNetwork(aInput_shape=list(), aModel_url=str(), aOutput_shape=int()):
    """
    Creates a deep learning neural network.
    """
    ### defining network architecture
    neural_net = tflow.keras.Sequential([
        thub.KerasLayer(handle=aModel_url), # input layer
        tflow.keras.layers.Dense(units=aOutput_shape, activation="softmax")]) # output layer
    ### setting metrics and optimization
    neural_net.compile(
        loss=tflow.keras.losses.CategoricalCrossentropy(), # error function
        optimizer=tflow.keras.optimizers.Adam(), # optimizer algorithm
        metrics=["accuracy"])
    ### building and returning network
    neural_net.build(input_shape=aInput_shape)
    return neural_net

#### Creating Callbacks

**Callbacks are event handler functions that are called at certain model training events.**  
The TensorBoard() callback saves a training log that helps in monitoring the training process.  
The EarlyStopping() callback halts training when a chosen metric stops improving.  

In [ ]:
### creating tensorboard callback
def callbackTensorboard():
    """
    Creates folder structure for TensorBoard logs.\
    Returns TensorBoard callback object.
    """
    log_root = Path("./logs")
    log_folder = log_root.joinpath(datetime.now().strftime("%Y%m%d-%H%M%S"))
    return tflow.keras.callbacks.TensorBoard(log_dir=log_folder)

In [ ]:
### creating early stopping callback
def callbackEarlyStopping():
    """
    Creates and returns an EarlyStopping callback object.
    """
    return tflow.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=5)

#### Training and Saving Neural Network

GPUs have limited amount of memory.  
The entire training dataset may not fit into GPU memory.  
To resolve this, we split our datasets into batches of ~32 tensors.  
The neural network sees only one batch at a time.

In [ ]:
### creating, training, and saving full model
recognition_model = createNetwork(aInput_shape=INPUT_SHAPE, aModel_url=MODEL_URL, aOutput_shape=OUTPUT_SHAPE)
recognition_model.fit(
    x=fTrain_dataset.batch(batch_size=32),
    epochs=NUM_EPOCHS,
    validation_data=fValid_dataset.batch(batch_size=32),
    validation_freq=1,
    callbacks=[callbackTensorboard(),callbackEarlyStopping()])
recognition_model.save(filepath="./models/fullModel")

In [ ]:
### creating, training, and saving reduced model
recognition_model = createNetwork(aInput_shape=INPUT_SHAPE, aModel_url=MODEL_URL, aOutput_shape=OUTPUT_SHAPE)
recognition_model.fit(
    x=rTrain_dataset.batch(batch_size=32),
    epochs=NUM_EPOCHS,
    validation_data=rValid_dataset.batch(batch_size=32),
    validation_freq=1,
    callbacks=[callbackTensorboard(),callbackEarlyStopping()])
recognition_model.save(filepath="./models/reducedModel")

#### Model Evaluation with TensorBoard

In [ ]:
### loading tensorboard
%tensorboard --logdir drive/MyDrive/ColabData/DogRecognition/training_logs

#### Loading Trained Model

In [ ]:
### loading trained model
recognition_loaded: tflow.keras.Model = tflow.keras.models.load_model(
    filepath="./models/reducedModel",
    custom_objects={"KerasLayer":thub.KerasLayer})

In [ ]:
### evaluating loaded model
recognition_loaded.evaluate(rValid_dataset.batch(batch_size=32))

## Visualizing Predictions

In [ ]:
### function plotting image, true label, and top predictions
def plotPredictions(aImage=numpy.ndarray([]), aLabel=numpy.ndarray([]), aPred=numpy.ndarray([])):
    """
    Plots an image of a dog, its true breed, and the top predicted breeds.
    """
    ### figure init
    pyplot.figure(figsize=(7,4.5))
    ### plotting image and true label
    image_plot = pyplot.subplot(1, 2, 1)
    image_plot.imshow(X=aImage)
    image_plot.set_title(label=unique_breeds[numpy.argmax(a=aLabel)], fontsize=10)
    image_plot.set_yticks(ticks=[])
    image_plot.set_xticks(ticks=[])
    ### plotting top 5 predictions
    top_indexes = numpy.argsort(a=aPred)[-5:][::-1]
    pred_plot = pyplot.subplot(1, 2, 2)
    pred_plot.bar(height=aPred[top_indexes], x=range(5))
    pred_plot.set_title(label="Top Five Predictions", fontsize=10)
    pred_plot.set_ylabel(ylabel="Confidence Levels", fontsize=10)
    pred_plot.set_xticks(ticks=range(5), labels=unique_breeds[top_indexes], rotation="vertical")
    ### layout and returning
    pyplot.tight_layout(h_pad=0.1)
    return

In [ ]:
### making predictions
valid_preds: numpy.ndarray = recognition_loaded.predict(x=rValid_dataset.batch(batch_size=32), verbose=True)
valid_preds.shape

In [ ]:
### visualizing select predictions
INDEX = 14
for index,item in enumerate(iterable=rValid_dataset.as_numpy_iterator()):
    if index == INDEX:
        plotPredictions(aImage=item[0], aLabel=item[1], aPred=valid_preds[INDEX])

Challenge: Create a confusion matrix of true labels versus predictions.